Create a ad hoc network having 10 nodes and a total number of 32 connections.
At each node, generate the data according to the following model:

$$ y_k(n) = \boldsymbol{\theta}_0^T \mathbf{x}_k(n) + \eta_k(n), 1 \leq k \leq 10, $$

where
* $ \boldsymbol{\theta}_0 \in \mathbb{R}^{60} $ is a constant vector, generated via $ N(0,1) $.
* all input vectors $ \mathbf{x}_k(n) \in \mathbb{R}^{60} $ are i.i.d generated via $ N(0,1) $.
* noise samples $ \eta_k(n) \in \mathbb{R}^1 $ are independently generated from zero mean Gaussians with variances corresponding to different signal-to-noise level, varying from 20-25 dBs in each node.

For the unknown vector estimation employ the following algorithms:
* combine-then-adapt diffusion APSM. Parameters: $ \mu_n = 0.5 \times M_n $, $ \epsilon_k = \sqrt 2 \sigma_k $, $ q = 20 $.
* adapt-then-combine LMS. Parameter: step size = 0.03.
* combine-then-adapt LMS. Parameter: step size = 0.03.
* noncooperative LMS.

Choose $ a_{mk} $ according to Metropolis rule:

$$ a_{mk} = \left\{
\begin{aligned}
&\frac{1}{max(n_k,n_m)}, &k \neq m,\ k,\ m\ are\ neighbors, \\
&0, &m \neq k, \\
&1-\sum_{i \neq k} a_{ik}, &m = k.
\end{aligned}
\right.
$$

Run 100 independent experiments and plot the average MSD per iteration in dBs:

$$ MSD(n) = 10 \log_{10} \left( \frac{1}{K} \sum_{k=1}^{K} \Vert \boldsymbol{\theta}_{k}(n) - \boldsymbol{\theta}_0 \Vert^2 \right). $$

* class networkController: generate $ \boldsymbol{\theta}_0 $ and send it to nodes, signal nodes to begin training, record training history, compute MSD.
* class node: generate input vector and noise, run algorithms, communicate with adjacent nodes.

In [1]:
# Import packages
import numpy as np
import scipy as sp
import matplotlib as plt

class networkController: generate $ \boldsymbol{\theta}_0 \in \mathbb{R}^{60} $ via $ N(0,1) $, compute true value according to sample, signal nodes to begin training, record training history, compute MSD.

In [40]:
# networkController
class networkController:
    
    def __init__(self):
        self.true_theta = np.zeros(60) # initialise theta0 in R^60
        self.energy = 0 # initialise energy level of signal
        self.nodes = [networkNode(i,self) for i in range(10)] # initialise nodelist
        self.count_nodes = len(self.nodes)
        
        # initialise training history
        self. caASPM_history = np.array(0)
        self.acLMS_history = np.array(0)
        self.caLMS_history = np.array(0)
        self.ncLMS_history = np.array(0)
        
        # algorithms
        self.algorithms = Algorithms(self)
    
    # generate theta0 via standard normal distribution
    def genTheta(self):
        self.true_theta = np.random.randn(60)
        self.energy = np.linalg.norm(self.true_theta)**2
        return
    
    # train models for iters iterations using selected algorithm
    def startTrain(self, iters: int, algorithm: str, initial_iters = 0, mu = 0.03):
        # clear theta vectors in all nodes
        self.clearThetas()
        
        # update adjacent vectors and convert node.adjacent_nodes from sets to lists
        for node in self.nodes:
            node.clearSampleHistory()
            node.adjacent_nodes = list(node.adjacent_nodes)
            node.updateArgs()
            node.mu = mu
            node.noise = np.sqrt(self.energy*np.power(10,-node.snr/10))
            
        # combine-then-adapt ASPM
        if algorithm == "caASPM":
            self.algorithms.caASPM(initial_iters, iters)
        
        # adapt-then-combine LMS
        elif algorithm == "acLMS":
            self.algorithms.acLMS(iters)
        
        # combine-then-adapt LMS
        elif algorithm == "caLMS":
            self.algorithms.caLMS(iters)
        
        # noncooperative LMS
        elif algorithm == "ncLMS":
            self.algorithms.ncLMS(iters)
        
        # convert node.adjacent_nodes from lists to sets
        for node in self.nodes:
            node.adjacent_nodes = set(node.adjacent_nodes)
        
        return
    
    # compute msd in dBs
    def computeMSD(self) -> float:
        msd = 0
        for node in self.nodes:
            msd += np.linalg.norm(self.true_theta-node.theta)**2
        msd = 10*np.log(msd/self.count_nodes)/np.log(10)
        return msd
    
    def computeY(self, x: np.array, noise: float) -> float:
        return np.dot(self.true_theta,x)+np.random.normal(0,noise)
    
    def networkInf(self):
        count_connections = (sum(node.count_nodes for node in self.nodes)-len(self.nodes))//2
        print("This network has %d nodes with %d connections."%(len(self.nodes),count_connections))
        print("The theta vector of this network is:\n",self.true_theta)
        print("The l^2 norm of the theta vector is:",np.sqrt(self.energy),"\n")
        return
    
    def clearThetas(self):
        for node in self.nodes:
            node.theta = np.zeros(60)
        return
    
    def clearHistory(self):
        self.caASPM_history = np.array(0)
        self.acLMS_history = np.array(0)
        self.caLMS_history = np.array(0)
        self.ncLMS_history = np.array(0)
        return

class algorithms: realise all training algorithms and store temporary variables.

In [41]:
class Algorithms:
    
    def __init__(self, controller: networkController):
        # network controller
        self.controller = controller
        
        # temporary arguments
        self.beta = 0
        self.M = 0
    
    # Algorithms
    # combine-then-adapt ASPM
    def caASPM(self, initial_iters: int, iters: int):
        for node in self.controller.nodes:
            node.popEarliestSample()
        
        for iter_num in range(initial_iters):
            # initial periods
            # sample
            for node in self.controller.nodes:
                node.sample()
                
            # compute psi
            for node in self.controller.nodes:
                node.psi = np.zeros(60)
                for i in range(node.count_nodes):
                    node.psi += node.adjacent_vector[i]*node.adjacent_nodes[i].theta
            
            # choose omega and compute theta
            for node in self.controller.nodes:
                node.theta = np.zeros(60)
                self.updatecaASPM(node,iter_num+1)
                node.mu = self.M/2
                node.theta *= node.mu
                node.theta += node.psi
            
            # compute msd and add into training history
            self.controller.caASPM_history = np.append(self.controller.caASPM_history,self.controller.computeMSD())
            
        # following periods: sliding window
        # sample
        for _ in range(iters):
            # sample
            for node in self.controller.nodes:
                node.sampleAndPop()
            
            # compute psi
            for node in self.controller.nodes:
                node.psi = np.zeros(60)
                for i in range(node.count_nodes):
                    node.psi += node.adjacent_vector[i]*node.adjacent_nodes[i].theta
            
            # compute theta
            for node in self.controller.nodes:
                node.theta = np.zeros(60)
                self.updatecaASPM(node,initial_iters)
                node.mu = self.M/2
                node.theta *= node.mu
                node.theta += node.psi
            
            # compute msd and add into training history
            self.recordHistory("caASPM")
            
        return
        
    # adapt-then-combine LMS
    def acLMS(self, iters: int):
        for _ in range(iters):
            # sample
            for node in self.controller.nodes:
                node.sampleAndPop()
            
            # compute error
            for node in self.controller.nodes:
                for i in range(node.count_nodes):
                    node.error[i] = node.adjacent_nodes[i].y[-1]-np.dot(node.theta,node.adjacent_nodes[i].x[-1])
            
            # compute psi
            for node in self.controller.nodes:
                node.psi = np.zeros(60)
                for i in range(node.count_nodes):
                    node.psi += node.adjacent_vector[i]*node.error[i]*node.adjacent_nodes[i].x[-1]
                node.psi *= node.mu
                node.psi += node.theta
            
            # compute new theta
            for node in self.controller.nodes:
                node.theta = np.zeros(60)
                for i in range(node.count_nodes):
                    node.theta += node.adjacent_vector[i]*node.adjacent_nodes[i].psi
            
            # compute msd and add into training history
            self.recordHistory("acLMS")
            
        return
        
    # combine-then-adapt LMS
    def caLMS(self, iters: int):
        for _ in range(iters):
            # sample
            for node in self.controller.nodes:
                node.sampleAndPop()
                
            # compute psi
            for node in self.controller.nodes:
                node.psi = np.zeros(60)
                for i in range(node.count_nodes):
                    node.psi += node.adjacent_vector[i]*node.adjacent_nodes[i].theta
                
            # compute error
            for node in self.controller.nodes:
                for i in range(node.count_nodes):
                    node.error[i] = node.adjacent_nodes[i].y[-1]-np.dot(node.psi,node.adjacent_nodes[i].x[-1])
                
            # compute new theta
            for node in self.controller.nodes:
                node.theta = np.zeros(60)
                for i in range(node.count_nodes):
                    node.theta += node.adjacent_vector[i]*node.error[i]*node.adjacent_nodes[i].x[-1]
                node.theta *= node.mu
                node.theta += node.psi
                
            # compute msd and add into training history
            self.recordHistory("caLMS")
        
        return
        
    # noncooperative LMS
    def ncLMS(self, iters: int):
        for _ in range(iters):
            # sample
            for node in self.controller.nodes:
                node.sampleAndPop()
                
            # compute error
            for node in self.controller.nodes:
                node.error[0] = node.y[-1]-np.dot(node.theta,node.x[-1])
                
            # compute new theta
            for node in self.controller.nodes:
                node.theta += node.mu*node.error[0]*node.x[-1]
            
            # compute msd and add into training history
            self.recordHistory("ncLMS")
            
        return
    
    # compute msd and add into training history
    def recordHistory(self, algorithm: str):
        current_msd = self.controller.computeMSD()
        
        # combine-then-adapt ASPM
        if algorithm == "caASPM":
            self.controller.caASPM_history = np.append(self.controller.caASPM_history,current_msd)
        
        # adapt-then-combine LMS
        elif algorithm == "acLMS":
            self.controller.acLMS_history = np.append(self.controller.acLMS_history,current_msd)
        
        # combine-then-adapt LMS
        elif algorithm == "caLMS":
            self.controller.caLMS_history = np.append(self.controller.caLMS_history,current_msd)
        
        # noncooperative LMS
        elif algorithm == "ncLMS":
            self.controller.ncLMS_history = np.append(self.controller.ncLMS_history,current_msd)
        
        return
    
    # compute beta and M in caASPM
    def updatecaASPM(self, node: networkNode, window_len: int):
        self.beta = 0
        self.M = 0
        for i in range(window_len):
            self.beta = node.y[i]-np.dot(node.x[i],node.psi)
            self.beta = np.maximum(0,abs(self.beta)-np.sqrt(2)*node.noise)*np.sign(self.beta)
            self.beta /= np.linalg.norm(node.x[i])**2
            node.theta += self.beta*node.x[i]
            self.M += (self.beta*np.linalg.norm(node.x[i]))**2
        node.theta /= window_len
        self.M /= window_len
        if np.linalg.norm(node.theta) != 0:
            self.M /= np.linalg.norm(node.theta)**2

class networkNode: add and delete adjacent nodes, generate sample, train the regressor $ \theta $, compute energy of the noise.

In [42]:
# networkNode
class networkNode:
    
    def __init__(self, num: int, controller: networkController):
        self.controller = controller
        self.num = num
        
        # main arguments
        self.theta = np.zeros(60) # initialise theta in R^60
        self.mu = 0.03 # initialise mu_k
        self.omega = np.zeros(0) # initialise convex combination arguments
        self.x = np.zeros((1,60)) # initialise sample history
        self.y = np.zeros(1) # initialise output history
        self.error = np.zeros(0) # initialise error vector
        self.psi = np.zeros(60) # initialise psi vector
        self.snr = np.random.uniform(20,25) # initialise signal-to-noise ratio
        self.noise = 0 # initialise noise
        
        # adjacent nodes
        self.adjacent_nodes = set([self]) # initialise adjacent nodelist
        self.count_nodes = 1 # count adjacent nodes
        self.adjacent_vector = np.zeros(0) # initialise matrix C and A = C
    
    # connect self and nodes in node_list
    # elements in node_list are numbers of node
    def addAdjacentNode(self, node_list: list):
        for node_num in node_list:
            node = self.controller.nodes[node_num]
            self.adjacent_nodes.add(node)
            node.adjacent_nodes.add(self)
            node.count_nodes = len(node.adjacent_nodes)
        self.count_nodes = len(self.adjacent_nodes)
        return
    
    # disconnect self and nodes in node_list
    # elements in node_list are numbers of node
    def removeAdjacentNode(self, node_list: list):
        for node_num in node_list:
            node = self.controller.nodes[node_num]
            self.adjacent_nodes.discard(node)
            node.adjacent_nodes.discard(self)
            node.count_nodes = len(node.adjacent_nodes)
        self.adjacent_nodes.add(self)
        self.count_nodes = len(self.adjacent_nodes)
        return
    
    def sample(self):
        sample = np.random.normal(0,1,(1,60))
        y = self.controller.computeY(sample[0],self.noise)
        self.x = np.append(self.x,sample,0)
        self.y = np.append(self.y,y)
        return
    
    def popEarliestSample(self):
        self.x = self.x[1:]
        self.y = self.y[1:]
        return
    
    def sampleAndPop(self):
        self.popEarliestSample()
        self.sample()
        return
    
    def updateArgs(self):
        # initialise a count_nodes-dimensionial error vector
        self.error = np.zeros(self.count_nodes)
        
        # select C and A = C
        self.adjacent_vector = np.zeros(self.count_nodes)
        for i in range(self.count_nodes):
            if self.adjacent_nodes[i].num != self.num:
                self.adjacent_vector[i] = 1/np.max([self.count_nodes,self.adjacent_nodes[i].count_nodes])
            else:
                mark = i
        self.adjacent_vector[mark] = 1-np.sum(self.adjacent_vector)
                
        return
    
    def clearSampleHistory(self):
        self.x = np.zeros((1,60))
        self.y = np.zeros(1)

In [43]:
controller = networkController()

In [44]:
controller.nodes[0].addAdjacentNode([1,4,6,8])
controller.nodes[1].addAdjacentNode([2,5,8])
controller.nodes[2].addAdjacentNode([3,7,9])
controller.nodes[3].addAdjacentNode([6,7,9])
controller.nodes[4].addAdjacentNode([2,5,7])
controller.nodes[5].addAdjacentNode([0,2,8,9])
controller.nodes[6].addAdjacentNode([1,2,7])
controller.nodes[7].addAdjacentNode([5,8,9])
controller.nodes[8].addAdjacentNode([2,3,4])
controller.nodes[9].addAdjacentNode([4,6,8])

In [45]:
controller.genTheta()
controller.networkInf()

This network has 10 nodes with 32 connections.
The theta vector of this network is:
 [ 0.79490206  0.05421345 -0.34943765  0.02288424 -0.80852947 -1.02801086
 -0.04725702  0.15044185  0.53455623  0.73202526  0.61414802 -0.08250928
 -0.9514484   0.18653716  0.14177171 -0.98196943  1.60734553 -0.02219069
  0.05186646 -0.62982341 -0.17167318  0.49713885 -0.81264513  0.6111494
  0.20798308 -0.94877098  0.10591816  0.08724529  1.31583473  0.27619668
 -0.8937843  -0.4350052   0.55740767 -0.09494761  1.56395083  0.06803868
 -2.02611442 -0.34932309 -1.35964161  0.73427506  1.43835872  0.4544722
  1.46120607  0.0857322  -0.46072379 -0.82636736  0.30230019 -0.62196663
 -1.54906655  0.40799434 -0.5440476  -0.22376627 -0.07162041  0.2570125
 -0.86947775 -1.28447114  0.64141523 -1.97502399 -0.51092861 -0.41665204]
The l^2 norm of the theta vector is: 6.243979157322803 



In [46]:
controller.startTrain(80,"caASPM",initial_iters = 20)
controller.startTrain(100,"acLMS")
controller.startTrain(100,"caLMS")
controller.startTrain(100,"ncLMS")

In [47]:
print(controller.caASPM_history)
print(controller.acLMS_history)
print(controller.caLMS_history)
print(controller.ncLMS_history)

[  0.          15.86534613  15.79055986  15.65353     15.44702253
  15.225162    15.00026144  14.73288459  14.43847247  14.15211679
  13.79536084  13.38505591  12.92020188  12.44009586  11.96365293
  11.41737775  10.87009765  10.32497251   9.76819438   9.21166365
   8.65182687   8.07827526   7.48600854   6.93131264   6.4070788
   5.87493865   5.35601305   4.87061869   4.38519751   3.89190038
   3.4206297    2.90227621   2.40617421   1.9121567    1.41581639
   0.93766404   0.49272068   0.05344707  -0.36387364  -0.75244488
  -1.11371099  -1.44968567  -1.75360853  -2.03679817  -2.3049497
  -2.5506967   -2.7979706   -3.04455366  -3.26035897  -3.48265218
  -3.75001925  -4.02152397  -4.28039833  -4.57110729  -4.89008465
  -5.19257365  -5.47809326  -5.76376465  -6.0351811   -6.28888727
  -6.53902484  -6.76503408  -6.99801162  -7.22601458  -7.43796064
  -7.64018173  -7.83592506  -8.00185273  -8.16722538  -8.35257578
  -8.52729132  -8.70363348  -8.8878887   -9.07541397  -9.23719146
  -9.3868823